# VertexAI Setup

In [78]:
import vertexai

vertexai.init()

# Client side function delcaration

Declare the `list_extensions` function, which returns the list of extensions currently declared in the previously initialised Vertex AI instance.

In [71]:
from vertexai.preview.extensions import Extension

def list_extensions():
    """Provides a list of Vertex AI extensions available to the user.

    Returns:
        list[tuple[str, str, str]]: A list of tuples where each tuple contains (in order) the
        display name, the resource name, and the creation time of an extension.
    """
    extensions_list = Extension.list()
    return [(
        e.display_name,
        e.resource_name,
        e.create_time.strftime("%m/%d/%Y, %H:%M:%S")
        ) for e in Extension.list()]

# Make client-side function schema interpretable by model

In [79]:
list_extensions_function = FunctionDeclaration.from_func(list_extensions)
list_extensions_function

name: "list_extensions"
description: "Provides a list of vertex ai extensions available to the user."
parameters {
  type_: OBJECT
  title: "list_extensions"
  description: "Provides a list of vertex ai extensions available to the user."
}

# Run the Agent with the "Function Calling" tool

In [80]:
import requests
import vertexai
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

model = GenerativeModel("gemini-1.5-flash-001")

user_prompt_content = Content(
    role="user",
    parts=[
        Part.from_text("Which Vetex AI extension do I have access to ?"),
    ],
)

support_tool = Tool(
    function_declarations=[list_extensions_function],
)

response = model.generate_content(
    user_prompt_content,
    generation_config=GenerationConfig(temperature=0),
    tools=[support_tool],
)


for function_call in response.candidates[0].function_calls:
    if function_call.name == "list_extensions":
        api_response = list_extensions()

    response = model.generate_content(
        [
            user_prompt_content,
            response.candidates[0].content,
            Content(
                parts=[
                    Part.from_function_response(
                        name=function_call.name,
                        response={"content": api_response},
                    ),
                ],
            ),
        ],
        tools=[support_tool],
    )

    print(response.text)

You have access to the following Vertex AI extension: Zefix. It was last updated on 01/22/2025, 11:08:20. 

